## Projektarbeit Mühle

### Imports

In [ ]:
import PySimpleGUI as psGui
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
from tensorflow import keras
import tensorflow as tf
import datetime as dt
import random
import sklearn.preprocessing as pre

### Class definitions

#### Environment

In [ ]:
class MillEnv(object):
    def __init__(self):
        self.isPlaying = 1
        self.gamePhase = 0
        self.inHand = [9,9]
        self.onBoard = [0,0]
        self.selcted = -1
        self.board = np.zeros((0,24))
        self.columns = [[0,1,2],
                        [3,4,5],
                        [6,7,8],
                        [9,10,11],
                        [12,13,14],
                        [15,16,17],
                        [18,19,20],
                        [21,22,23],
                        [0,9,21],
                        [3,10,18],
                        [6,11,15],
                        [1,4,7],
                        [16,19,22],
                        [8,12,17],
                        [5,13,20],
                        [2,14,23],
                        ]
    def makeMove(self, player, pos):
        valid = False
        if self.gamePhase == 0:
            if self.board[pos] == 0:
                self.board[pos] = player
                valid = True
                self.isPlaying = -self.isPlaying
                self.inHand[player] -= 1
            if np.array(self.inHand).sum() == 0:
                self.gamePhase = 1
            return valid
        if self.gamePhase == 1:
            if self.board[pos] == 1 * player:
                valid = True
                self.selcted = pos
                self.gamePhase = 2
            return valid
        return valid
    def getBoard(self):
        return copy.deepcopy(self.board)
    def isFinisched(self):
        for column in self.columns:
            columnSum = self.board[column[0]] + self.board[column[1]] + self.board[column[2]]
            if abs(columnSum) == 3:
                return self.board[column[0]]
        if self.board.all():
            return 2
        return 0
    def reset(self):
        self.board = np.zeros((0,24))
        self.isPlaying = 1
    def getSummary(self, player):
        numTwoPlayerActual = 0
        numTwoPlayerOpponent = 0
        numFreeRows = 0
        for column in self.columns:
            columnSum = self.board[column[0]] + self.board[column[1]] + self.board[column[2]]
            if columnSum == -2 * player:
                numTwoPlayerOpponent += 1
            elif columnSum  == 2 * player:
                numTwoPlayerActual += 1
            elif columnSum == 0 and self.board[column[0]] == self.board[column[1]]:
                numFreeRows += 1
        return numTwoPlayerActual, numTwoPlayerOpponent, numFreeRows, self.isFinisched() * player